In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = './data/'

import glob

folders = [f for f in glob.glob(path + "**/", recursive=False)]

for f in folders:
    print(f)

./data/train/
./data/val/
./data/test/


In [4]:
labels = pd.read_csv("final_results.txt", delimiter='\t', names = ['label', 'id', 'name', 'confidence'])
labels = labels.drop_duplicates(subset = 'name', keep='first')

match = pd.read_csv('labellist.txt', delimiter='    ', names = ['catagory', 'name'])

df = pd.merge(match, labels, on=['name'])
df = df.drop(['label', 'confidence'], axis = 1)

/home/liqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [6]:
path = './imagenet-a/'

for i in range(100):
    os.rename(path + df['catagory'][i], path + str(df['id'][i]))

In [7]:
# for i in range(100):
for i in range(100):
    for idx, f in enumerate(glob.glob(path + str(i) + "/**", recursive=False)):
        os.rename(f, path + str(i) + '/' + str(i) + '_' + str(idx + 1100) + '.jpg')